<a href="https://colab.research.google.com/github/Manikanta-Reddyy/machinelearningproblem2/blob/main/case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import time
import pandas as pd

API_KEY = "YOUR_ETHERSCAN_API_KEY"  # Replace with your key
WALLET_SHEET_URL = "https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/export?format=csv"
OUTPUT_CSV = "risk_scores.csv"

# Compound V2 cToken addresses (Mainnet)
cTOKENS = [
    "0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5", "0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643",
    "0x39AA39c021dfbaE8faC545936693aC917d5E7563", "0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9",
    "0xC11b1268C1A384e55C48c2391d8d480264A3A7F4", "0x70e36f6BF80a52b3B46b3aF8e106CC0ed743E8e4",
    "0x35A18000230DA775CAc24873d00Ff85BccdeD550", "0x6C8c6b02E7b2BE14d4fA6022Dfd6d75921D90E4E",
    "0xB3319f5D18Bc0D84dD1b4825Dcde5d5f7266d407", "0xe65cdB6479BaC1e22340E4E755fAE7E509EcD06c",
    "0xFAce851a4921ce59e912d19329929CE6da6EB0c7", "0x95b4eF2869eBD94BEb4eEE400a99824BF5DC325b",
    "0x4B0181102A0112A2ef11AbEE5563bb4a3176c9d7", "0x80a2AE356fc9ef4305676f7a3E2Ed04e12C33946",
    "0x041171993284df560249B57358F931D9eB7b925D", "0x7713DD9Ca933848F6819F38B8352D9A15EA73F67"
]
BATCHES = [cTOKENS[:8], cTOKENS[8:]]  # Split into two batches

# Compound V2 Event Signatures
EVENT_SIGS = {
    "Borrow": "0x13ed6866d4e1ee6da46f845c46d7e54120883d75c5ea9a2dacc1c4ca8984ab80",
    "RepayBorrow": "0x1a2a22cb034d26d1854bdc6666a5b91fe25efbbb5dcad3b0355478d6f5c362a1",
    "LiquidateBorrow": "0x298637f684da70674f26509b10f07ec2fbc77a335ab1e7d6215a4b2484d8bb52"
}

def get_event_count(wallet: str, event_sig: str, batch: list) -> int:
    """Fetch event count for a wallet from Etherscan."""
    topic1 = '0' * 24 + wallet[2:]  # Pad address to 32 bytes
    url = (
        f"https://api.etherscan.io/api?module=logs&action=getLogs"
        f"&fromBlock=0&toBlock=latest&address={','.join(batch)}"
        f"&topic0={event_sig}&topic1={topic1}&apikey={API_KEY}"
    )
    try:
        res = requests.get(url, timeout=10).json()
        return len(res["result"]) if res["status"] == "1" else 0
    except:
        return 0

def calculate_risk_score(borrows: int, repays: int, liquidations: int) -> int:
    """Compute risk score (0-1000) from transaction features."""
    L = min(1, liquidations / 5.0)  # Normalize liquidations (cap at 5)
    B = min(1, borrows / 100.0)     # Normalize borrows (cap at 100)
    ratio = borrows / (repays + 1)  # Avoid division by zero
    R = min(1, ratio / 10.0)        # Normalize ratio (cap at 10)
    risk_index = 0.5 * L + 0.3 * B + 0.2 * R
    return int(risk_index * 1000)

# Main Execution
wallets = pd.read_csv(WALLET_SHEET_URL)["wallet_id"].tolist()
with open(OUTPUT_CSV, "w") as f:
    f.write("wallet_id,score\n")
    for i, wallet in enumerate(wallets):
        wallet = wallet.strip().lower()
        if len(wallet) != 42 or not wallet.startswith("0x"):
            continue  # Skip invalid addresses

        counts = {"Borrow": 0, "RepayBorrow": 0, "LiquidateBorrow": 0}
        for event_name, event_sig in EVENT_SIGS.items():
            for batch in BATCHES:
                counts[event_name] += get_event_count(wallet, event_sig, batch)
                time.sleep(0.2)  # Rate limiting

        score = calculate_risk_score(
            counts["Borrow"], counts["RepayBorrow"], counts["LiquidateBorrow"]
        )
        f.write(f"{wallet},{score}\n")
        print(f"Processed {i+1}/{len(wallets)}: {wallet} = {score}")

Processed 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3 = 0
Processed 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988 = 0
Processed 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c = 0
Processed 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9 = 0
Processed 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae = 0
Processed 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416 = 0
Processed 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078 = 0
Processed 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596 = 0
Processed 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc = 0
Processed 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7 = 0
Processed 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22 = 0
Processed 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668 = 0
Processed 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872 = 0
Processed 14/103: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f = 0
Processed 15/103: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669 = 0
Processed 16/103: 0x22d7510588d90e